## 전처리 함수

In [1]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from time import sleep

org_df = pd.read_csv("../data/orginazation_label.csv", encoding='latin1')  # org csv파일
stock_df = pd.read_csv('../data/nasdaq_stocks.csv', encoding='latin1')  # stock list csv 파일
rep_df = pd.read_csv('C:/projectnasdaq/nasdaq_stocks_refine_total.csv', encoding='latin1')  #대표단어로 정리된 csv 파일
test_df = pd.read_csv('C:/example/test_sample.csv')  # 제대로 매핑이 됐는지 확인하기 위한 정답지 csv (1000개의 데이터를 직접 매핑한 csv)
org_df = org_df.head(1000)

stock_df.drop(columns=["sector", "industry"], inplace=True)  # stock.csv에서 필요없는 컬럼 제거

org_df = org_df[['label_id', 'news_id', 'word', 'label']]
org_df['pk'] = 0
org_df['sim_word'] = org_df['word']  # 유사도용 org 컬럼 생성
org_df['rep_word'] = org_df['word']  # 대표단어용 org 컬럼 생성

stock_df = stock_df[['pk', 'symbol', 'name']]
stock_df['sim_stock'] = stock_df['name']  # 유사도용 stock 컬럼 생성
org_df['token'] = 0

# ORG단어와 종목명 소문자화
org_df['sim_word'] = org_df['sim_word'].str.lower()
stock_df['sim_stock'] = stock_df['sim_stock'].str.lower()
org_df['rep_word'] = org_df['rep_word'].str.lower()

# 정규식 전처리
org_df['sim_word'] = org_df['sim_word'].apply(lambda x: re.sub('[^a-zA-Z\d]', ' ', str(x)).strip())
stock_df['sim_stock'] = stock_df['sim_stock'].apply(lambda x: re.sub('[^a-zA-Z\d]', ' ', str(x)).strip())
org_df['sim_word'] = org_df['sim_word'].apply(lambda x: re.sub(r'\d+', ' ', str(x)).strip())
stock_df['sim_stock'] = stock_df['sim_stock'].apply(lambda x: re.sub(r'\d+', ' ', str(x)).strip())

# 토큰 단어 먼저 불용어한 후 토큰화하기 위해서 먼저 전처리
stop_words_token = {'bloomberg', 'inc', 'ltd', 'company', 'corporation', 'corp'}
# ORG 단어들을 토큰화
org_df['sim_word'].replace(stop_words_token, '', regex=True, inplace=True)
org_df['token'] = org_df['sim_word'].str.split(" ")

token_list = org_df['token']
# token 컬럼에 있는 띄어쓰기 지우기
token_list = token_list.apply(lambda x: [i for i in x if i != "" and i != " "])
token_list = token_list.apply(lambda x: x[:3])
org_df['token'] = token_list

# 불용어 리스트"
# ORG에 대한 불용어 리스트
stop_words_org = {'bloomberg', 'inc', 'corp', 'ltd', 'company', 'corporation', 'composite', 'financial', 'plc',
                  'international'}
# stock_df['sim_stock'](종목명)에 대한 불용어 리스트
stop_words_stock = {'inc', 'corp', 'ltd', 'company', 'corporation', 'fund', 'composite', 'financial', 'incorporated',
                    'index', 'plc', 'international'}

org_df['sim_word'].replace(stop_words_org, '', regex=True, inplace=True)
stock_df['sim_stock'].replace(stop_words_stock, '', regex=True, inplace=True)

# org와 stock의 젤 오른쪽에 붙은 'co', 공백 제거
org_df['sim_word'] = org_df['sim_word'].str.rstrip("co"" ")
stock_df['sim_stock'] = stock_df['sim_stock'].str.rstrip("co"" ")
# 젤 왼쪽에 붙은 공백 제거
org_df['sim_word'] = org_df['sim_word'].str.lstrip(" ")

# 매핑하기 위해서 리스트에 담아주기
org_li = org_df['sim_word']
stock_li = stock_df['sim_stock']
rep_li = org_df['rep_word']

## 맵핑 코드

In [58]:
# ORG와 stocklist의 교집합이 존재시 맵핑
for token_num, token in enumerate(token_list):
    for stock_num, stock in enumerate(stock_li):
        stock_list = stock.split(" ")
        together = set(stock_list) & set(token)
        # ORG와 stock의 교집합이 2개일 경우 맵핑
        if len(together) == 2:
            org_df['pk'][token_num] = stock_df['pk'][stock_num]
            break
        # ORG와 stock의 교집합이 3개 이상일 경우 맵핑
        elif len(together) >= 3:
            org_df['pk'][token_num] = stock_df['pk'][stock_num]
            break

C:\Users\q1035\anaconda3\envs\aip\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\q1035\anaconda3\envs\aip\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [59]:
# 유사도 계산 ( 80 이상일 경우 맵핑 )
for org_num, org in enumerate(org_li):
    for stock_num, stock in enumerate(stock_li):
        if fuzz.ratio(org, stock) > 80:
            org_df['pk'][org_num] = stock_df['pk'][stock_num]
            break

C:\Users\q1035\anaconda3\envs\aip\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [60]:
# 크롤링 - 유사도가 80-85사이인 ORG들 따로 리스트에 담기
result = []
for org_num, org in enumerate(org_li):
    for stock_num, stock in enumerate(stock_li):
        if 80 <= fuzz.ratio(org, stock) <= 85:  # j가 org리스트
            if org not in result:
                result.append(org)
                break
# 리스트에 담은 org를 데이터프레임으로 변환
result_pd = pd.DataFrame(result, columns=['org'])
result_pd = result_pd['org']

In [62]:
# Google사이트에서 티커 정보 크롤링
stock_symbol = stock_df['symbol']
# 유사도가 80-85 사이인 ORG 단어들을 사용해서 티커 정보 크롤링
for num_org, org in enumerate(result_pd):
    # 옵션 생성 -> 크롤링 할 때 chrome 창 숨기기
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome('C:\chromedriver\chromedriver.exe', options=options)

    base_url = 'https://www.google.com/search?q='
    tail = org + "+" + "stock"
    url = base_url + tail
    driver.get(url)
    driver.implicitly_wait(time_to_wait=10)
    try:
        stock = driver.find_element_by_xpath(
            '//*[@id="rcnt"]/div[1]/div/div/div[3]/div[1]/div/div[2]/div/div/div/div[2]/div[1]').text
        driver.quit()
        sleep(1)
        stock = stock.split(" ")[1]
        for num_symbol, symbol in enumerate(stock_symbol):
            # ORG 검색 결과의 Ticker와 stock의 Ticker가 같을 경우
            if stock == symbol:
                # ORG단어와 stock을 맵핑
                print('org : ', org, ', stock : ', stock, ', symbol : ', symbol)
                org_df.loc[org_df['sim_word'] == org, 'pk'] = stock_df['pk'][num_symbol]
                break
    # 검색 결과가 없으면 해당 org는 non_stock 처리
    except NoSuchElementException:
        org_df.loc[org_df['sim_word'] == org, 'pk'] = 0
        print(org, '-->nonstock')
        continue

moderna therapeutics -->nonstock
org :  amazon , stock :  AMZN , symbol :  AMZN
org :  united airlines holdings , stock :  UAL , symbol :  UAL
org :  goldman sachs , stock :  GS , symbol :  GS
org :  s p global , stock :  SPGI , symbol :  SPGI
eskom holdings -->nonstock
russian energy -->nonstock
org :  new york times , stock :  NYT , symbol :  NYT
org :  banco santander sa , stock :  SAN , symbol :  SAN
group -->nonstock
org :  bp , stock :  BP , symbol :  BP
epic systems -->nonstock
rhs -->nonstock
tenev -->nonstock
org :  horizon therapeutics , stock :  HZNP , symbol :  HZNP
org :  vertex pharmaceuticals , stock :  VRTX , symbol :  VRTX
times -->nonstock
org :  citigroup , stock :  C , symbol :  C
org :  vodacom group , stock :  VOD , symbol :  VOD
cdc group -->nonstock
org :  mtn group , stock :  MTN , symbol :  MTN
crystal vision holdings -->nonstock
vitol group -->nonstock
saudi energy -->nonstock
org :  eog resources , stock :  EOG , symbol :  EOG
palantir technologies -->nonsto

In [ ]:
# 유사도 계산 ( 유사도 90 이상일 경우 맵핑 )
for org_num, org in enumerate(org_li):
    for stock_num, stock in enumerate(stock_li):
        if fuzz.ratio(org, stock) >= 90:
            org_df['pk'][org_num] = stock_df['pk'][stock_num]
            break

In [ ]:
# 대표단어 csv 파일 이용해서 맵핑
rep_li_a = rep_li.str.split(" ")  # org단어
l4 = rep_df['name_a'].str.split(",")  # 핵심단어로 정리된 stock_list
# 대표단어 csv파일에서 대표단어 수가 2개일 때 첫번째 단어가 있으면 넘어가서 두번째 자리에 다음 단어가 있는지 확인 후 맵핑
for i, j in enumerate(rep_li_a):  # 전처리 안한 org단어
    for k, l in enumerate(l4):  # 대표단어로 정리된 stock_list
        if len(l) == 1:
            if l[0] in j:
                org_df['pk'][i] = rep_df['pk'][k]
                break
        elif len(l) == 2:  # 대표단어가 2개로 된 단어일 때
            if l[0] in j:  # 대표단어의 첫번째 단어가 org단어에 있으면
                found = j.index(l[0])  # 대표단어의 첫번째 단어와 일치하는 org단어의 인덱스 위치 번호
                try:
                    search = found + 1  # stocklist의 첫번째 단어가 org에 포함됐을때 그 다음 단어
                    search_found = j[search]  # org의 (+1을 한) 다음 단어에 해당
                    if l[1] == search_found:
                        org_df['pk'][i] = rep_df['pk'][k]
                        break
                except IndexError:
                    continue
        elif len(l) == 3:
            if l[0] in j:
                try:
                    search_found = j[j.index(l[0]) + 1]
                    if l[1] == search_found:
                        two_found = j[j.index(l[0]) + 2]
                        if l[2] == two_found:
                            org_df['pk'][i] = rep_df['pk'][k]
                            break
                except IndexError:
                    continue

In [ ]:
# ORG와 stock이 완전 일치시 맵핑
for org_num, org in enumerate(org_li):
    for stock_num, stock in enumerate(stock_li):
        if org == stock:
            org_df['pk'][org_num] = stock_df['pk'][stock_num]
            break

In [ ]:
# 정말 stock이 아니라고 생각하는 단어가 포함되면 non_stock 처리
stop_word_nonstock = {'university', 'federal reserve', 'committee', 'bank of', 'national institute', 'department',
                      'united nation', 'commission', 'world bank'}
for org_num, org in enumerate(org_li):
    for stopword_num, stopword in enumerate(stop_word_nonstock):
        if stopword in org:
            org_df['pk'][org_num] = 0
            break

In [ ]:
# bank of america가 들어가면 맵핑
banks = {'bank of america'}
for org_num, org in enumerate(org_li):
    for bank_num, bank in enumerate(banks):
        if bank in org:
            org_df['pk'][org_num] = 860

In [ ]:
# org 데이터중 공백란인 데이터에 pk=291이 매핑돼서 pk=291인 데이터들 전부 non stock처리
org_df.loc[org_df['pk'] == 291, 'pk'] = 0

In [ ]:
org_df.drop(columns=["sim_word", "rep_word", "token"], inplace=True)
stock_df.drop(columns=["sim_stock"], inplace=True)

In [ ]:
print('전체개수 : ', len(org_df))
print('분류된개수 : ', len(org_df[org_df['pk'] > 0]))
print('분류된 비율 : ', len(org_df[org_df['pk'] > 0]) / len(org_df))

In [ ]:
merge_left = pd.merge(org_df, stock_df, how='left', left_on='pk', right_on='pk')
merge_left

In [50]:
address = "C:/example/finish17.csv"

# 정확도 계산
merge_left.to_csv(address)
df_test = pd.read_csv("C:/example/test_sample.csv", encoding='latin1')
df2_test = pd.read_csv(address, encoding='latin1')
# stock->stock으로 잘 분류한 애들 True로 출력
address_2 = 'C:/example/df3_2.csv'

df3 = df_test['symbol'] == df2_test['symbol']
df3.to_csv(address_2)

# True인 개수 -> stock-stock으로 잘 분류한 개수 구하기 위해서 새로 df가져오기
df4 = pd.read_csv(address_2)

print('non_stock 개수 :', len(test_df[test_df['pk'] == 'Non_stock']))
print('stock 개수 :', len(test_df[test_df['pk'] != 'Non_stock']['pk']))
print('stock -> stock으로 분류한 개수 :', len(df4[df4['symbol'] == True]))
print('non -> non으로 분류한 개수 :', 1000 - len(df2_test[df2_test['pk'] > 0]['pk']))
print('총 맵핑 수 :', len(df2_test[df2_test['pk'] > 0]['pk']))
print('정확도 :', ((1000 - len(df2_test[df2_test['pk'] > 0]['pk'])) + len(df4[df4['symbol'] == True])) / 1000 * 100)

non_stock 개수 : 796
stock 개수 : 204
stock -> stock으로 분류한 개수 : 200
non -> non으로 분류한 개수 : 763
총 맵핑 수 : 237
정확도 : 96.3
